In [3]:
#Import Packages
import numpy as np
import pandas as pd

In [27]:
#Import Datasets

#Transcript dataset we created with our measures from the transcripts
transcript_features = pd.read_csv("./../data/transcript_features.csv", index_col="Video_Number")
print(transcript_features.head())

#Hand collected data over the duration of the project
#I already cleaned this one in excel prior to saving as a csv
video_features = pd.read_csv("./../data/video_features.csv", index_col="Video_Number")
print(video_features.head())

              Total_Words  Filler_Words  Unique_Words  Avg_Word_Freq  \
Video_Number                                                           
1                    1077           118           296       3.638514   
2                     790            76           273       2.893773   
3                     527            48           209       2.521531   
4                     933           111           306       3.049020   
5                     615            56           214       2.873832   

              Med_word_Freq   Entropy       TTR  
Video_Number                                     
1                       1.0  7.017286  0.274838  
2                       1.0  6.966307  0.345570  
3                       1.0  6.710372  0.396584  
4                       1.0  6.954690  0.327974  
5                       1.0  6.628696  0.347967  
             Date_Recorded  Hours_CI_Students  Hours_Reading  Hours_Talking  \
Video_Number                                                      

In [19]:
#Let's check to see if there are any duplicates in the transcript_features data

transcript_features[transcript_features.duplicated()] 

#Whoops! Checking the transcripts, it looks like videos 56 and 57 have the same transcript
#I accidentally copied in the contents of 56 into 57 (I copied and pasted manually all transcripts from youtube into txt files)

,Total_Words,Filler_Words,Unique_Words,Avg_Word_Freq,Med_word_Freq,Entropy,TTR
Video_Number,,,,,,,
57,662,64,223,2.96861,1.0,6.804833,0.336858


In [31]:
#Fixed and re ran the previous two jupyter notebooks and now let's check again in the next cell

transcript_features[transcript_features.duplicated()] 

#Great! Glad we caught that. 

,Total_Words,Filler_Words,Unique_Words,Avg_Word_Freq,Med_word_Freq,Entropy,TTR
Video_Number,,,,,,,


In [53]:
#Let's check to see if there are any duplicates in the video_features data

video_features[video_features.duplicated(subset=video_features.columns.difference(["Date_Recorded"]))]

#Nope! This is expected, since the date values are unique I checked for duplicates on the data for every column but that one

,Date_Recorded,Hours_CI_Students,Hours_Reading,Hours_Talking,Hours_Native_Media,Time_Recorded,Length_Minutes
Video_Number,,,,,,,


In [45]:
#Now let's joing the two datasets

data = pd.merge(video_features, transcript_features, on="Video_Number", how="inner")

print(len(video_features), len(transcript_features), len(data)) 

#All three are the same length -> data joined well (since data is 1-1)

112 112 112


In [63]:
#Let's make sure there are no null values:

print("Number of Null Values:") 
print(data.isna().sum())

#Great, no null values!

Number of Null Values:
Date_Recorded         0
Hours_CI_Students     0
Hours_Reading         0
Hours_Talking         0
Hours_Native_Media    0
Time_Recorded         0
Length_Minutes        0
Total_Words           0
Filler_Words          0
Unique_Words          0
Avg_Word_Freq         0
Med_word_Freq         0
Entropy               0
TTR                   0
dtype: int64


In [69]:
#Now let's check to make sure all the data types are what they should be

data.dtypes

#All these make sense! I'm going to convert Time_Recorded into an HH:MM variable and Date_Recorded into a datetime variable

Date_Recorded          object
Hours_CI_Students     float64
Hours_Reading         float64
Hours_Talking         float64
Hours_Native_Media    float64
Time_Recorded          object
Length_Minutes        float64
Total_Words             int64
Filler_Words            int64
Unique_Words            int64
Avg_Word_Freq         float64
Med_word_Freq         float64
Entropy               float64
TTR                   float64
dtype: object

In [79]:
#Converting strings into the proper datetime variables I want them to be in
data["Time_Recorded"] = pd.to_datetime(data["Time_Recorded"], format="%I:%M %p").dt.strftime("%H:%M")
data["Date_Recorded"] = pd.to_datetime(data["Date_Recorded"], format="%m/%d/%y")

In [81]:
#Now let's look at the five number summary for the data and make sure everything "makes sense" according to the measure

data.describe()

#Things I look for - negative values!!, 0's in columns they shouldn't be in, min/max values in reasonable ranges
#Everything looks great!

#Note: I am not looking at columns to exclude, new features, or anything like that- although this can be considered part of cleaning.
#    I want to keep that in the analysis file, just to make it easier for people to see my train of thought (because I anticipate not many
#    will read into the files outside of that one)

,Date_Recorded,Hours_CI_Students,Hours_Reading,Hours_Talking,Hours_Native_Media,Length_Minutes,Total_Words,Filler_Words,Unique_Words,Avg_Word_Freq,Med_word_Freq,Entropy,TTR
count,112,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.0,112.000000,112.000000
mean,2024-12-23 22:42:51.428571392,0.832589,0.511161,0.388393,3.064732,9.589107,910.732143,71.383929,276.723214,3.210958,1.0,6.891041,0.317795
min,2024-10-28 00:00:00,0.000000,0.000000,0.000000,0.000000,5.120000,459.000000,23.000000,171.000000,2.390625,1.0,6.346722,0.227094
25%,2024-11-24 18:00:00,0.000000,0.000000,0.000000,2.000000,7.420000,664.250000,55.750000,230.750000,2.932008,1.0,6.778526,0.290957
50%,2024-12-22 12:00:00,0.000000,0.250000,0.125000,3.000000,8.835000,839.500000,67.500000,266.500000,3.143714,1.0,6.880027,0.318095
75%,2025-01-22 06:00:00,2.000000,0.750000,0.500000,4.000000,11.397500,1047.000000,85.250000,310.500000,3.437182,1.0,7.006264,0.341074
max,2025-02-19 00:00:00,3.750000,3.250000,3.000000,10.250000,16.050000,2037.000000,140.000000,470.000000,4.403465,1.0,7.389269,0.418301
std,NaN,1.092763,0.709800,0.604822,1.463807,2.777628,326.888442,24.384032,63.047131,0.467117,0.0,0.168120,0.044874


In [83]:
#Next is to make sure all variable names are valid. I noticed I accidentally left the w in "Med_word_Freq" lowercase
#Other than that, all variable names are good (and I thought about this before). Let's make the quick change!

data.rename(columns={"Med_word_Freq": "Med_Word_Freq"}, inplace=True)

print(data.columns)

Index(['Date_Recorded', 'Hours_CI_Students', 'Hours_Reading', 'Hours_Talking',
       'Hours_Native_Media', 'Time_Recorded', 'Length_Minutes', 'Total_Words',
       'Filler_Words', 'Unique_Words', 'Avg_Word_Freq', 'Med_Word_Freq',
       'Entropy', 'TTR'],
      dtype='object')


In [87]:
#Let's look at our beautifully clean dataset in the face! (Or rather, head)

data.head()

,Date_Recorded,Hours_CI_Students,Hours_Reading,Hours_Talking,Hours_Native_Media,Time_Recorded,Length_Minutes,Total_Words,Filler_Words,Unique_Words,Avg_Word_Freq,Med_Word_Freq,Entropy,TTR
Video_Number,,,,,,,,,,,,,,
1,2024-10-28,2.5,0.50,0.5,2.0,11:33,13.25,1077,118,296,3.638514,1.0,7.017286,0.274838
2,2024-10-29,1.5,0.00,1.0,3.5,10:54,9.18,790,76,273,2.893773,1.0,6.966307,0.345570
3,2024-10-30,3.0,1.00,1.0,2.0,21:50,7.03,527,48,209,2.521531,1.0,6.710372,0.396584
4,2024-10-31,2.5,1.25,0.0,2.5,23:36,11.92,933,111,306,3.049020,1.0,6.954690,0.327974
5,2024-11-01,1.5,1.00,0.0,1.5,23:27,8.27,615,56,214,2.873832,1.0,6.628696,0.347967


In [89]:
#Finally, let's export this data into a csv file and finish up the cleaning process!

data.to_csv("./../data/clean_data.csv")